In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import util
import random
import embeddings
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import word_tokenize
import ast
import pickle


# torch.manual_seed(1)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
df = pd.read_csv("./Final_Data/collections_all_science_out-temp_lda.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4943 entries, 0 to 4942
Data columns (total 44 columns):
resource_id                4943 non-null object
title                      4943 non-null object
url                        4943 non-null object
created_at                 4943 non-null object
updated_at                 4943 non-null object
creator_id                 4943 non-null object
modifier_id                4943 non-null object
original_creator_id        4943 non-null object
original_content_id        4943 non-null object
parent_content_id          4943 non-null object
publish_date               28 non-null object
publish_status             4943 non-null object
narration                  4162 non-null object
description                4943 non-null object
content_format             4943 non-null object
content_subformat          4943 non-null object
answer                     0 non-null float64
metadata                   3031 non-null object
taxonomy                   4561 n

In [3]:
len(df.collection_id.unique())


1393

In [4]:
columns_required = ["collection_id","resource_id","sequence_id","title","description",
                    "Summarization","clean","clean_matrix","lda_topics"
                   ]

In [5]:
data = df[columns_required]
data.head(1)

,collection_id,resource_id,sequence_id,title,description,Summarization,clean,clean_matrix,lda_topics
0,001ed871-f958-4f85-a638-6219d47b3935,9dad3a1d-6f8b-4ef6-bc2c-b207d8d2ce1f,2,Table of Contents - Google Docs,Table of Contents - Google Docs,"Atom FCNElement C, M WSChemical Rxn FCNChem an...","['atom', 'fcnelement', 'c', 'wschemical', 'rxn...","[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1...","[(0, 0.0029411765), (1, 0.0029411765), (2, 0.0..."


In [6]:
data.sort_values(by=["collection_id","sequence_id"])

,collection_id,resource_id,sequence_id,title,description,Summarization,clean,clean_matrix,lda_topics
0,001ed871-f958-4f85-a638-6219d47b3935,9dad3a1d-6f8b-4ef6-bc2c-b207d8d2ce1f,2,Table of Contents - Google Docs,Table of Contents - Google Docs,"Atom FCNElement C, M WSChemical Rxn FCNChem an...","['atom', 'fcnelement', 'c', 'wschemical', 'rxn...","[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1...","[(0, 0.0029411765), (1, 0.0029411765), (2, 0.0..."
1,001ed871-f958-4f85-a638-6219d47b3935,d796ed55-67db-41b6-ae29-1f55a51b8fb6,5,The Atom,"In this interactive activity from ChemThink, l...",Each atom is composed of subatomic particles: ...,"[u'atom', u'composed', u'subatomic', u'particl...","[(0, 3), (3, 1), (11, 1), (15, 1), (16, 1), (1...","[(0, 0.00086206896), (1, 0.00086206896), (2, 0..."
2,001ed871-f958-4f85-a638-6219d47b3935,15aad96a-9bb1-4039-ab5c-094a1027de29,8,Atoms flashcards | Quizlet,Vocabulary words for Atomic Structure. Include...,Atoms Flashcards | Quizlet SearchCreateLog inS...,"[u'atom', u'flashcard', u'quizlet', u'searchcr...","[(0, 2), (23, 2), (25, 1), (44, 2), (45, 1), (...","[(0, 0.16006736), (1, 0.00069444446), (2, 0.00..."
3,00247455-2bac-4152-af3f-0b33860e7e15,c3284b10-b6d4-46e2-848b-d7ce79aace59,1,Presentation: Rise of Facism,Overview of the rise and spread of Fascism.,"Presentation: Rise of Facism, Overview of the ...","['presentation', 'rise', 'facism', 'overview',...","[(105, 1), (106, 2), (107, 1), (108, 1), (109,...","[(0, 0.24888113), (1, 0.0038461536), (2, 0.003..."
5,00413ae4-2088-4319-97b1-8216a4fee1e7,1da33e99-92df-406f-aabb-d6e20a4bdbb2,2,Reading: The First Law of Thermodynamics,An introductory reading on the first law of th...,Citizenship 2018Business of the Year Why Trust...,"[u'citizenship', u'2018business', u'year', u't...","[(41, 1), (55, 1), (66, 2), (110, 1), (113, 1)...","[(0, 0.00031847137), (1, 0.00031847137), (2, 0..."
4,00413ae4-2088-4319-97b1-8216a4fee1e7,cf6df015-c58c-4236-aeb7-c6abf574eed0,3,Reading: First Law of Thermodynamics in Depth,An in-depth reading from NASA on the First Law...,Many observations of real gases have shown tha...,"[u'many', u'observation', u'real', u'gas', u's...","[(41, 1), (55, 1), (113, 1), (114, 1), (115, 1...","[(0, 0.00060975616), (1, 0.00060975616), (2, 0..."
6,00413ae4-2088-4319-97b1-8216a4fee1e7,b4e5fdbd-f08a-4dad-8ef3-a671dffe5f22,4,Reading: First Law and Types of Processes,An in-depth reading on the First Law that also...,The internal energy of a system can also incre...,"[u'internal', u'energy', u'system', u'also', u...","[(2, 2), (41, 1), (55, 1), (90, 1), (113, 1), ...","[(0, 0.010950349), (1, 0.00044247785), (2, 0.0..."
14,008fe8b8-abad-4f91-9aa9-184117b8fd44,fa8ff713-e5b5-4174-916f-a8341f428a50,1,Mitosis,"Cell Biology Animation presents Mitosis, an ed...","Mitosis, Cell Biology Animation presents Mitos...","[u'mitosis', u'cell', u'biology', u'animation'...","[(17, 1), (49, 1), (193, 1), (250, 1), (329, 1...","[(0, 0.0045454544), (1, 0.0045454544), (2, 0.0..."
17,008fe8b8-abad-4f91-9aa9-184117b8fd44,82a6fd6f-dd70-4eb3-826a-c8c36b04c875,2,Meiosis,"Cell Biology Animation presents Meiosis, an ed...","Meiosis, Cell Biology Animation presents Meios...","[u'meiosis', u'cell', u'biology', u'animation'...","[(17, 1), (49, 1), (193, 1), (250, 1), (329, 1...","[(0, 0.0045454544), (1, 0.0045454544), (2, 0.0..."
9,008fe8b8-abad-4f91-9aa9-184117b8fd44,a5909836-ae80-4b4f-b0e7-f4120b1b815d,3,Cell Division,Find out about cell reproduction and division ...,"BBC - Error 404 : Not Found <p style=""position...","[u'bbc', u'error', u'404', u'found', u'p', u's...","[(49, 1), (64, 1), (71, 1), (90, 1), (96, 2), ...","[(0, 0.00034965036), (1, 0.028160488), (2, 0.0..."


In [7]:
# loading word2vec and function to retuen word embeddings

word2vec = Word2Vec.load('./Final_Data/Word2Vec_100dim_science')

def Word2doc(sentence,dim):
    words=sentence.strip().split()
    emb=np.zeros(dim)
    for word in words:
        if(word in word2vec.wv.vocab):
            emb = np.add(emb,word2vec[word])
    return list(emb/len(words))       


In [8]:
Doc2vec_model= Doc2Vec.load("./Final_Data/doc2vec_100dim_science.model")
 

def Doc2vec(sentence,dim):
    emb = np.zeros(dim) 
    emb += Doc2vec_model.infer_vector(word_tokenize(sentence.strip().lower()))
    return list(emb)   


In [9]:
data["Word_embeddings"] = data.Summarization.apply(lambda x: Word2doc(x,100))
data["Doc_embeddings"] = data.Summarization.apply(lambda x: Doc2vec(x,100))

/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
def stringlist_to_list(x):
    return (ast.literal_eval(x))


In [11]:
data["lda_topics"] = data.lda_topics.apply(lambda x: stringlist_to_list(x))
len(data.lda_topics)

/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


4943

In [16]:
collections = data.collection_id.values
print("coll_le:",collections)
data.info()

coll_le: ['001ed871-f958-4f85-a638-6219d47b3935'
 '001ed871-f958-4f85-a638-6219d47b3935'
 '001ed871-f958-4f85-a638-6219d47b3935' ...
 'ffdaab48-66e0-4adc-a073-7b1028079f48'
 'ffdaab48-66e0-4adc-a073-7b1028079f48'
 'ffee59cd-6bc3-4ad2-8137-d37270d3a529']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4943 entries, 0 to 4942
Data columns (total 11 columns):
collection_id      4943 non-null object
resource_id        4943 non-null object
sequence_id        4943 non-null int64
title              4943 non-null object
description        4943 non-null object
Summarization      4943 non-null object
clean              4943 non-null object
clean_matrix       4943 non-null object
lda_topics         4943 non-null object
Word_embeddings    4943 non-null object
Doc_embeddings     4943 non-null object
dtypes: int64(1), object(10)
memory usage: 424.9+ KB


In [17]:
collections = (set(collections))

In [18]:
frame = [] #[cids,rids,sids,summ's,lda's]
for i in collections:
    cids = []
    rids = []
    sids = []
    summs = []
    lda_topic = []
    descs = []
    titles = []
    word_embeddings = []
    doc_embeddings = []
    df_temp = data[data.collection_id == i] 
    for index,row in df_temp.iterrows():
        cids.append(row["collection_id"])
        rids.append(row["resource_id"])
        sids.append(row["sequence_id"])
        descs.append(row["description"])
        summs.append(row["Summarization"])
        lda_topic.append(row["lda_topics"])
        titles.append(row["title"])
        word_embeddings.append(row["Word_embeddings"])
        doc_embeddings.append(row["Doc_embeddings"])        
    frame.append([cids,rids,sids,descs,summs,lda_topic,titles,word_embeddings,doc_embeddings,1])
    

In [19]:
columns = ["collection_ids","resource_ids","sequence_ids","descriptions",
           "Summarizations","lda_topics","titles","Word_embeddings","Doc_embeddings","label"]
final_dataframe = pd.DataFrame(frame, columns =columns)

In [20]:
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393 entries, 0 to 1392
Data columns (total 10 columns):
collection_ids     1393 non-null object
resource_ids       1393 non-null object
sequence_ids       1393 non-null object
descriptions       1393 non-null object
Summarizations     1393 non-null object
lda_topics         1393 non-null object
titles             1393 non-null object
Word_embeddings    1393 non-null object
Doc_embeddings     1393 non-null object
label              1393 non-null int64
dtypes: int64(1), object(9)
memory usage: 108.9+ KB


In [29]:
def generate_random_collections(df, size, avg_col_length = 7):
    total_df_size = df.shape[0]
    
    collections = []
    for i in range(size):
        collection = {}
        collection["collection_ids"] = []
        collection["sequence_ids"] = []
        collection["resource_ids"] = []
        collection["titles"] = []
        collection["descriptions"] = []
        collection["Summarizations"] = []
        collection["lda_topics"] = []
        collection["Word_embeddings"] = []
        collection["Doc_embeddings"] = []
        for j in range(avg_col_length):
            pick = random.randint(0,total_df_size-1)
            row = df.iloc[pick,:]
            while(row['collection_id'] in collection['collection_ids']):
                pick = random.randint(0,total_df_size-1)
                row = df.iloc[pick,:]
            collection['collection_ids'].append(row['collection_id'])
            collection["sequence_ids"].append(row["sequence_id"])
            collection["resource_ids"].append(row["resource_id"])
            collection["titles"].append(row["title"])
            collection["descriptions"].append(row["description"])
            collection["Summarizations"].append(row["Summarization"])
            collection["lda_topics"].append(row["lda_topics"])
            collection["Word_embeddings"].append(row["Word_embeddings"])
            collection["Doc_embeddings"].append(row["Doc_embeddings"])    
            
            
#         collection["collection_ids"] = np.array(collection["collection_ids"])
#         collection["sequence_ids"] = np.array(collection["sequence_ids"])
#         collection["resource_ids"] = np.array(collection["resource_ids"])
#         collection["titles"] = np.array(collection["titles"])
#         collection["descriptions"] = np.array(collection["descriptions"])
#         collection["Summarizations"] = np.array(collection["Summarizations"])
#         collection["lda_topics"] = list(collection["lda_topics"])
#         collection["Word_embeddings"] = np.array(collection["Word_embeddings"])
#         collection["Doc_embeddings"] = np.array(collection["Doc_embeddings"])
        collections.append(collection)
    
    return collections


In [34]:
rand = generate_random_collections(data,800,avg_col_length = 5)

In [35]:
random_data = pd.DataFrame(rand)
random_data["label"]=0
random_data.head(1)

,Doc_embeddings,Summarizations,Word_embeddings,collection_ids,descriptions,lda_topics,resource_ids,sequence_ids,titles,label
0,"[[-0.0008301076013594866, -0.00268635246902704...",[Energy in a Roller Coaster Ride 2006 WGBH Edu...,"[[0.05599650915931253, 0.0602357615442837, -0....","[f7d2faa9-fe08-478f-89eb-05505b939066, 2ea00c2...",[Click play on this interactive to see the cha...,"[[(0, 0.0031250003), (1, 0.0031250003), (2, 0....","[b8bc4ba6-edb5-49b7-b95b-9f8c15fe4cec, f51517f...","[4, 9, 22, 2, 2]","[Roller Coaster, Electromagnetic Waves, What ...",0


In [24]:
type(data.lda_topics)

pandas.core.series.Series

In [25]:
type(random_data.lda_topics)

pandas.core.series.Series

In [27]:
final_dataframe.to_csv("./Final_Data/Valid_collections_science1.csv", index=False)
random_data.to_csv("./Final_Data/Random_Data_science1.csv", index=False)

In [ ]:
random_data.shape

In [ ]:
final_dataframe.shape

In [ ]:
final_dataframe.head(1)

In [ ]:
random_data.head(1)

In [ ]:
type(final_dataframe.Word_embeddings[0])

In [36]:
pickle_out = open("./Final_Data/Valid_processed_science1.pickle","wb")
pickle.dump(final_dataframe, pickle_out)
pickle_out.close()

pickle_out = open("./Final_Data/Invalid_processed_science1.pickle","wb")
pickle.dump(random_data, pickle_out)
pickle_out.close()